# An Introduction to WISER, Part 3: Neural Networks

In this part of the tutorial, we use the probabilistic labels of the generative model to train a state-of-the-art recurrent neural network. We will use a bi-LSTM-CRF to improve the generalization of the link-HMM

To begin, we will enable logging to keep track of the training logs. You may observe this in the terminal used to open this notebook.

In [1]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import pickle 

with open('output/generative/link_hmm/train_data_750.p', 'rb') as f:
    dat = pickle.load(f)
with open('output/generative/link_hmm/dev_data.p', 'rb') as f:
    dev = pickle.load(f)

In [6]:


for instance in dev:
    unary = instance['unary_marginals'].as_tensor(instance['unary_marginals'].get_padding_lengths())
    tokens = [t for t in instance['tokens']]
    for marg in unary:
        if marg[2] >=0.4:
            print(marg)
#     print(unary)
#     printed = False
#     for i in range(len(unary)):
#         if unary[i][2] > 0.4:
#             print(tokens[i].text)
#             printed = True
#         elif printed:
#             print()
#             printed = False
        
#     for tok, marg in zip(instance['tokens'], instance['unary_marginals'].as_tensor(instance['unary_marginals'].get_padding_lengths())):
        
#         if  marg[1] > 0.5:
#             print (tok)
#         print(tok, marg)

tensor([0.2101, 0.0570, 0.7330])
tensor([0.1921, 0.0212, 0.7867])
tensor([0.5448, 0.0344, 0.4208])
tensor([0.0984, 0.0928, 0.8088])
tensor([0.0661, 0.0614, 0.8726])
tensor([0.0281, 0.0256, 0.9464])
tensor([0.1296, 0.1207, 0.7497])
tensor([0.1013, 0.0930, 0.8057])
tensor([0.0679, 0.0616, 0.8706])
tensor([0.0288, 0.0257, 0.9455])
tensor([0.0657, 0.0609, 0.8735])
tensor([0.0279, 0.0254, 0.9467])
tensor([0.0657, 0.0609, 0.8735])
tensor([0.0279, 0.0254, 0.9467])
tensor([0.0663, 0.0608, 0.8729])
tensor([0.0286, 0.0253, 0.9461])
tensor([0.0979, 0.0926, 0.8094])
tensor([0.0656, 0.0612, 0.8733])
tensor([0.0275, 0.0254, 0.9471])
tensor([0.0254, 0.0249, 0.9497])
tensor([0.0268, 0.0252, 0.9480])
tensor([0.0636, 0.0610, 0.8754])
tensor([0.0984, 0.0928, 0.8088])
tensor([0.0660, 0.0614, 0.8726])
tensor([0.0281, 0.0256, 0.9464])
tensor([0.1014, 0.0922, 0.8064])
tensor([0.0682, 0.0610, 0.8708])
tensor([0.0294, 0.0254, 0.9452])
tensor([0.0663, 0.0608, 0.8729])
tensor([0.0286, 0.0253, 0.9461])
tensor([0.

tensor([0.0656, 0.0612, 0.8733])
tensor([0.0275, 0.0254, 0.9471])
tensor([0.0254, 0.0249, 0.9497])
tensor([0.0268, 0.0252, 0.9480])
tensor([0.0636, 0.0610, 0.8754])
tensor([0.0266, 0.0250, 0.9484])
tensor([0.0254, 0.0249, 0.9497])
tensor([0.0268, 0.0252, 0.9480])
tensor([0.0636, 0.0610, 0.8754])
tensor([0.1296, 0.1207, 0.7497])
tensor([0.1013, 0.0930, 0.8057])
tensor([0.0679, 0.0616, 0.8706])
tensor([0.0288, 0.0257, 0.9455])
tensor([0.1301, 0.1206, 0.7493])
tensor([0.1019, 0.0929, 0.8052])
tensor([0.0685, 0.0615, 0.8700])
tensor([0.0295, 0.0256, 0.9449])
tensor([0.0984, 0.0928, 0.8088])
tensor([0.0661, 0.0614, 0.8726])
tensor([0.0281, 0.0256, 0.9464])
tensor([0.1326, 0.1199, 0.7474])
tensor([0.1035, 0.0926, 0.8040])
tensor([0.0692, 0.0614, 0.8694])
tensor([0.0293, 0.0256, 0.9451])
tensor([0.0989, 0.0927, 0.8083])
tensor([0.0667, 0.0613, 0.8720])
tensor([0.0288, 0.0255, 0.9457])
tensor([0.1833, 0.1666, 0.6502])
tensor([0.1619, 0.1451, 0.6931])
tensor([0.1362, 0.1208, 0.7430])
tensor([0.

tensor([0.1168, 0.1008, 0.7824])
tensor([0.0725, 0.0619, 0.8657])
tensor([0.0311, 0.0258, 0.9431])
tensor([0.2203, 0.1983, 0.5814])
tensor([0.2037, 0.1811, 0.6152])
tensor([0.1778, 0.1564, 0.6658])
tensor([0.1461, 0.1275, 0.7264])
tensor([0.1078, 0.0935, 0.7987])
tensor([0.0719, 0.0620, 0.8661])
tensor([0.0304, 0.0259, 0.9437])
tensor([0.0989, 0.0927, 0.8084])
tensor([0.0667, 0.0613, 0.8720])
tensor([0.0288, 0.0255, 0.9457])
tensor([0.0657, 0.0609, 0.8735])
tensor([0.0279, 0.0254, 0.9467])
tensor([0.0984, 0.0928, 0.8088])
tensor([0.0660, 0.0614, 0.8726])
tensor([0.0281, 0.0256, 0.9464])
tensor([0.0984, 0.0928, 0.8088])
tensor([0.0661, 0.0614, 0.8726])
tensor([0.0281, 0.0256, 0.9464])
tensor([0.0989, 0.0927, 0.8084])
tensor([0.0667, 0.0613, 0.8720])
tensor([0.0288, 0.0255, 0.9457])
tensor([0.1296, 0.1207, 0.7497])
tensor([0.1013, 0.0930, 0.8057])
tensor([0.0679, 0.0616, 0.8706])
tensor([0.0288, 0.0257, 0.9455])


In [24]:
for instance in dev:
    unary = instance['unary_marginals'].as_tensor(instance['unary_marginals'].get_padding_lengths())
    tokens = [t for t in instance['tokens']]
       
    printed = False
    for i in range(len(unary)):
        if unary[i][2] > 0.4:
            print(tokens[i].text)
            printed = True
        elif printed:
            print()
            printed = False
        
#     for tok, marg in zip(instance['tokens'], instance['unary_marginals'].as_tensor(instance['unary_marginals'].get_padding_lengths())):
        
#         if  marg[1] > 0.5:
#             print (tok)
#         print(tok, marg)

Academy
Award
-

Golden
Globe
Award

Screen
Actors
Guild
Award

Academy
Awards

Emmy
Awards

Grammy
Awards

Golden
Globe
Award
for
Best
Actor

Primetime
Emmy
Award

Primetime
Emmy
Award

Emmy
Awards

Tony
Award
for
Best
Leading
Actor

Tony
Awards

Emmy
Awards

Primetime
Emmy
Awards

Academy
Awards

Academy
Awards

Golden
Globe
Award
for
Best
Actor

Emmy
Award

Olivier
Award
for
Best
Actor

Tony
Awards
:

Tony
Award
for
Best
Actor

Academy
Award

Award
for
Best
Supporting
Actor

Award
for
Best
Actor

Golden
Globe
Award

Academy
Award
for
Best
Supporting
Actor

Golden
Globe
Award
-

Academy
Award
for
Best
Supporting
Actor

Nobel
Prize

Academy
Award
-

Academy
Award
for
Best
Supporting
Actor

Academy
Award
for
Best
Supporting
Actor

Academy
Award
for
Best
Supporting
Actor

Volpi
Cup
for
Best
Actor
at
the
Venice
Film
Festival

Academy
Award

Primetime
Emmy
Awards

Tony
Award

Mark
Twain
Prize

Academy
Awards

Golden
Globe
Award
for
Best
Actor

Britannia
Humanitarian
Award

Primetime
Emmy


In [19]:
print(dat[1]['tokens'])

TextField of length 57 with text: 
 		[He, has, received, numerous, accolades, ,, including, two, Academy, Awards, ,, a, Golden, Globe,
		Award, ,, the, Cecil, B., DeMille, Award, ,, the, Golden, Lion, ,, the, AFI, Life, Achievement,
		Award, ,, Presidential, Medal, of, Freedom, ,, and, has, been, nominated, for, six, BAFTA, Awards,
		,, four, Primetime, Emmy, Awards, and, four, Screen, Actors, Guild, Awards, .]
 		and TokenIndexers : {'tokens': 'SingleIdTokenIndexer'}


## Training a Neural Network

To train your bi-LSTM-CRF network, you first need to define an [AllenNLP configuration file](https://github.com/allenai/allennlp/blob/master/tutorials/getting_started/walk_through_allennlp/configuration.md). These files provide a versatile way of defining neural networks, detailing all the model parameters. For this tutorial, we have already provided a configuration file, but feel free to edit it or use your own

We have provided the wrapper method ``train_discriminative_model`` on ``wiser.discriminative`` that you can use to . 
To train a bi-LSTM-CRF on the link HMM's probabilistic output, you need to call the method and pass it in the location of the generative models' output. You may also specify the cuda device and random seed for training. 

During training, tou should be able to observe logs on the terminal used to open this notebook. TODO: Mention that they shouldn't expect an evolution of train scores

In [ ]:
from wiser.discriminative import train_discriminative_model

train_discriminative_model(
    train_data_path='output/generative/link_hmm/train_data.p',
    dev_data_path='output/generative/link_hmm/dev_data.p',
    test_data_path='output/generative/link_hmm/test_data.p',
    train_config_path='training_config/tutorial.jsonnet',
    output_path='output/discriminative/link_hmm',
    cuda_device=-1)

## Predicting Tags

In [ ]:
# TODO